<a href="https://colab.research.google.com/github/VonFaust/BM-calibration/blob/main/Amitim_BM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mplfinance

In [ ]:
pip install yfinance

In [ ]:
pip install python-exchangeratesapi

In [ ]:
pip install pandas_datareader

In [ ]:
######  import and functions ######
import yfinance as yf
import pandas as pd
import numpy as np
#from google.colab import files
import matplotlib.pyplot as plt
import pandas_datareader.data as web
#"av-forex" api_key='RG5CHBTEPQ4821U9WYS'
#quandl.ApiConfig.api_key ='RG7YpiKTqZ9iXmsGECG_8V8'
from datetime import datetime
import matplotlib.ticker as mtick



def fx_israel(start,end,provider='fred',ticker='CCUSSP02ILM650N',df='USDILS'):
    dfname = df
    
    df = web.DataReader(ticker, provider, start, end)
    df.rename(columns={ticker:'Price'},inplace =True)
    df.sort_index(ascending=True)
    df['Return']= df['Price'].pct_change()
    df['Moving Average 12 month'] = df['Return'].rolling(12).mean()
    df['Moving Std 12 month'] = df['Return'].rolling(12).std()
    df['Rolling Yearly Return'] = (df['Price'].iloc[::] / df['Price'].shift(12)-1)
    df[dfname] = df['Price'] / df['Price'][0]
    df.rename(columns={'Price': dfname + " Price"},inplace =True)
    df.name = dfname
    return df




def get_bm_data_github(link,df):
    dfname = df
    
    link = str(link)
    df = str(df)
    df = pd.read_csv(link, error_bad_lines=False)
    df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df.sort_index(ascending=True)
    df['Return']= df['Price'].pct_change()
    df['Moving Average 12 month'] = df['Return'].rolling(12).mean()
    df['Moving Std 12 month'] = df['Return'].rolling(12).std()
    df['Rolling Yearly Return'] = (df['Price'].iloc[::] / df['Price'].shift(12)-1)
    df[dfname] = df['Price'] / df['Price'][0]
    df.rename(columns={'Price': dfname + " Price"},inplace =True)
    df.name = dfname
    return df

def get_interest_data_github(link,df):
    dfname = df
    
    link = str(link)
    df = str(df)
    df = pd.read_csv(link, error_bad_lines=False)
    df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df.sort_index(ascending=True)
    df['Return']= df['Price']
    df['Moving Average 12 month'] = df['Price'].rolling(12).mean()
    df['Moving Std 12 month'] = df['Price'].rolling(12).std()
    df['Rolling Yearly Return'] = (df['Price'].iloc[::] / df['Price'].shift(12)-1)
    df[dfname] = df['Price'] / df['Price'][0]
    df.rename(columns={'Price': dfname + " Price"},inplace =True)
    df.name = dfname
    return df


def get_data_yahoo(ticker,start,end,df,interval='1mo'):
    #define the ticker symbol
    dfname = df
    
    tickerSymbol = str(ticker)
    tickerData = yf.Ticker(tickerSymbol)
    df = tickerData.history(interval ='1mo', start = start, end = end)
    #drop the columns, keep only date and price
    df = df.drop(df.columns[[0, 1, 2, 5,6, 4]], axis=1)
    #Calculate Return and insert to the DF
    df.sort_index(ascending=True)
    df['Return']= df['Close'].pct_change()
    df['Moving Average 12 month'] = df['Return'].rolling(12).mean()
    df['Moving Std 12 month'] = df['Return'].rolling(12).std()
    #Yearly returns ( (i/i-12) - 1) in monthly increments
    df['Rolling Yearly Return'] = (df['Close'].iloc[::] / df['Close'].shift(12)-1)
    df[dfname] = df['Close'] / df['Close'][0]
    df.rename(columns={'Close': dfname + " Price"},inplace =True)
    df.name = dfname
    return df



def get_tlvex_data_github(link,df):
    dfname = df
    
    link = str(link)
    df = pd.read_csv(link, error_bad_lines=False)
    ###### determine if end of the month
    df['If end month'] = 0
    for i in range(0,len(df['Date'])-1):
        if (df['Date'].iloc[i][3:5]) != (df['Date'].iloc[i+1][3:5]):
              df['If end month'].iloc[i] = 1  
        else:
              #df['If end month'].iloc[i] = 0
                pass
    #Drop the irelevant rows
    df = df[df['If end month'] == 1]
    df.drop(columns='If end month',inplace=True)
    #set index as date
    df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df.sort_index(ascending=True)
    #calculations of return
    df['Return']= df['Price'].pct_change()
    df['Moving Average 12 month'] = df['Return'].rolling(12).mean()
    df['Moving Std 12 month'] = df['Return'].rolling(12).std()
    df['Rolling Yearly Return'] = (df['Price'].iloc[::] / df['Price'].shift(12)-1)
    df[dfname] = df['Price'] / df['Price'][0]
    df.rename(columns={'Price': dfname + " Price"},inplace =True)
    df.name = dfname
    return df    


def get_df_name(df):
    ######to retrive a df name
    name =[x for x in globals() if globals()[x] is df][0]
    return name


def df_sclicer(df,start,end='none'):
    #p = get_df_name(df)
    p = df.name
    dfname = p + '_' + start
    start = pd.Timestamp(start)
    if end != 'none':
        end = pd.Timestamp(end)
        dfname = pd.DataFrame(df.loc[start:end][df.columns[0]])
    else:
        dfname = pd.DataFrame(df.loc[start:][df.columns[0]])
    dfname['Return']= dfname[[dfname.columns[0]]].pct_change()
    dfname['Moving Average 12 month'] = dfname['Return'].rolling(12).mean()
    dfname['Moving Std 12 month'] = dfname['Return'].rolling(12).std()
    dfname['Rolling Yearly Return'] = (dfname[df.columns[0]].iloc[::] / dfname[dfname.columns[0]].shift(12)-1)
    dfname[p] = dfname[dfname.columns[0]] / dfname[dfname.columns[0]][0]
    dfname.name = p# + '_' + str(start)
    return dfname



def df_single_plot(df,type='stock',save = 'no'):
    
    #name =get_df_name(df)
    name = df.name
   
    
    if type == 'stock':
        fig, axs = plt.subplots(5,figsize=(7, 15),constrained_layout= True)
        axs[0].plot(df[name+' Price'],color='orange')
        axs[0].set_title(name + ' - Price')


        axs[1].plot(df['Return'],color='blue')
        axs[1].set_title(name + " - Return (monthly)")
        axs[1].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

        axs[2].plot(df['Moving Average 12 month'],color='mediumpurple')
        axs[2].set_title(name + " - Moving Average 12 month")
        axs[2].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

        axs[3].plot(df['Moving Std 12 month'],color='black')
        axs[3].set_title(name + " - Moving Std 12 month")
        axs[3].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

        axs[4].plot(df['Rolling Yearly Return'],color='green')
        axs[4].set_title(name + " - Rolling Yearly Return")
        axs[4].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    else:
        fig, axs = plt.subplots(4,figsize=(7, 15),constrained_layout= True)
        
        axs[0].plot(df[name+' Price'],color='orange')
        axs[0].set_title(name + ' - Rate')
        axs[0].yaxis.set_major_formatter(mtick.PercentFormatter())
        
        axs[1].plot(df['Moving Average 12 month'],color='mediumpurple')
        axs[1].set_title(name + " - Moving Average 12 month")
        axs[1].yaxis.set_major_formatter(mtick.PercentFormatter())

        axs[2].plot(df['Moving Std 12 month'],color='black')
        axs[2].set_title(name + " - Moving Std 12 month")
        axs[2].yaxis.set_major_formatter(mtick.PercentFormatter())

        axs[3].plot(df['Rolling Yearly Return'],color='green')
        axs[3].set_title(name + " - Rolling Yearly Rate")
        axs[3].yaxis.set_major_formatter(mtick.PercentFormatter())
        
    if save != 'no':
        plt.savefig(name +' plot.png')
    else:
        pass
    

def df_two_plots(df1,df2,save= 'no'):
    name1 = df1.name
    name2 = df2.name

    fig, axs = plt.subplots(2, 5,figsize=(20, 7),constrained_layout= True)

    axs[0,0].plot(df1[name1+' Price'],color='orange')
    axs[0,0].set_title(name1 + ' - Price')
    
    axs[0,1].plot(df1['Return'],color='blue')
    axs[0,1].set_title(name1 + " - Return (monthly)")
    axs[0,1].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    axs[0,2].plot(df1['Moving Average 12 month'],color='mediumpurple')
    axs[0,2].set_title(name1 + " - Moving Average 12 month")
    axs[0,2].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    axs[0,3].plot(df1['Moving Std 12 month'],color='black')
    axs[0,3].set_title(name1 + " - Moving Std 12 month")
    axs[0,3].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    axs[0,4].plot(df1['Rolling Yearly Return'],color='green')
    axs[0,4].set_title(name1 + " - Rolling Yearly Return")
    axs[0,4].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
                                       
    axs[1,0].plot(df2[name2 +' Price'],color='orange')
    axs[1,0].set_title(name2 + ' - Price')
                                       
    axs[1,1].plot(df2['Return'],color='blue')
    axs[1,1].set_title(name2 + " - Return (monthly)")
    axs[1,1].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    axs[1,2].plot(df2['Moving Average 12 month'],color='mediumpurple')
    axs[1,2].set_title(name2 + " - Moving Average 12 month")
    axs[1,2].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    axs[1,3].plot(df2['Moving Std 12 month'],color='black')
    axs[1,3].set_title(name2 + " - Moving Std 12 month")
    axs[1,3].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    axs[1,4].plot(df2['Rolling Yearly Return'],color='green')
    axs[1,4].set_title(name2 + " - Rolling Yearly Return")
    axs[1,4].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    if save != 'no':
        plt.savefig(name1+'_'+name2 +'.png')
    else:
        pass
    
    
#def download_df_excel(df):
  #Download Excel
  #df.to_excel(f'{df}.xls') 
  #files.download(f'{df}.xls')



In [ ]:
TA125 = get_data_yahoo('^TA125.TA','1991-12-31','2021-3-21','TA125')
USDILS = fx_israel('1991-12-31','2021-3-21')
MSCI = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/World%20BM.csv','MSCI')
CSUSHPINSA  = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/CSUSHPINSA.csv','CSUSHPINSA')
FNCOTR  = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/FNCOTR%20Index.csv','FNCOTR')
GDDUWI  = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/GDDUWI%20Index.csv','GDDUWI')
H0A0 = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/H0A0%20Index.csv','H0A0')
ISBRATE = get_interest_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/ISBRATE%20%20Index.csv','ISBRATE')
LGDRTRUH = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/LGDRTRUH%20Index.csv','LGDRTRUH')
LUACOAS = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/LUACOAS%20%20Index.csv','LUACOAS')
MIMUAWON = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/MIMUAWON%20Index.csv','MIMUAWON')
RMSN = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/RMSN.csv','RMSN')
NDDUWI = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/NDDUWI%20Index.csv','NDDUWI')
RMZN = get_bm_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/rmz.csv','RMZN')
TB60 = get_tlvex_data_github('https://raw.githubusercontent.com/VonFaust/BM-calibration/main/Telbond60%20index.csv','TB60')
EURILS = get_data_yahoo('EURILS=X','2003-01-31','2021-3-21','EURILS')

tickers = [TA125,USDILS,MSCI,CSUSHPINSA,FNCOTR,GDDUWI,H0A0,ISBRATE,LGDRTRUH,LUACOAS,MIMUAWON,RMSN,NDDUWI,RMZN,TB60,EURILS]
#tickers1 = [TA125,USDILS]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
### Get the min/max date from whole BMs data
for i in tickers: 
    print (get_df_name(i) , i.index.min())
    print (get_df_name(i) , i.index.max())

In [ ]:
####### single plot of all BMs
for i in tickers: 
    if i.name == 'ISBRATE':
        df_single_plot(i,'Interest rate')
    else:
        df_single_plot(i,'stock')

In [ ]:
####### Describe of all BMS
for i in tickers:
    name = i.name+'_desc'
    name = i.describe()
    name.to_csv(i.name+'_desc.csv')

In [ ]:
###### Corr Matrix for dates

#new = choose_col_corr(5,'2008-02-03','2021-01-01')
#plt.plot(new)   

tickers2 = [TA125,USDILS,MSCI,FNCOTR,GDDUWI,H0A0,ISBRATE,LGDRTRUH,LUACOAS,MIMUAWON,RMSN,NDDUWI,RMZN,TB60,EURILS]

start = '2008-02-03'
end = '2020-01-01'
num = 4
df = pd.DataFrame()


for i in tickers2:
    try:
        df1 = df_sclicer(i,start,end)
        #print(df1.name, df1.size())
        df[i.name +'_'+ str(df1.columns[num])] = df1[df1.columns[num]].values
    except:
        i.sort_values(by = ['Date'],ascending=True , inplace = True)
        df1 = df_sclicer(i,start,end)
        #print(df1.name, df1.size())
        df[i.name +'_'+ str(df1.columns[num])] = df1[df1.columns[num]].values
        
#df.corr()

df.corr().to_csv('corr '+str(start)+'_'+str(end)+'_.csv')




In [ ]:
####### Plot 2 BM side by side + slicing the relevant dates

TA1252 = df_sclicer(TA125,'1998-01-01')
MSCI2 = df_sclicer(MSCI,'1998-01-01')
df_two_plots(MSCI2,TA1252)